In [11]:
from openai import OpenAI
import os

In [12]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [13]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [14]:
print(chat_completion)

ChatCompletion(id='chatcmpl-8NMFRq79hdHZqIoICmPmI0yi5mMAC', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1700577757, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))
